**notebook um für bildausschnitte aus bilderen eines datensatzes mit dino attentionmaps zwischen zu speichern, damit das nicht immer direkt im training geschehen muss**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!git clone https://github.com/cocodataset/cocoapi

!cd /content/cocoapi/PythonAPI && make

In [ ]:
 !pip install timm
 
 !git clone https://github.com/Moldazien/BA.git

In [ ]:
import os
os.chdir('/content/BA')

In [ ]:
import os
import sys
import argparse
import cv2
import random
import colorsys
import requests
from io import BytesIO

import skimage.io
from skimage.measure import find_contours
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms as pth_transforms
import numpy as np
from PIL import Image

import utils
import vision_transformer as vits

from pycocotools.coco import COCO

In [ ]:
arch = 'vit_base' #'vit_small''vit_base'
patch_size = 8

output_dir = '/content/gdrive/MyDrive/Testing/first_test_dino'
    

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# build model
model = vits.__dict__[arch](patch_size=patch_size, num_classes=0)
for p in model.parameters():
    p.requires_grad = False
model.eval()
model.to(device)

#laden der pretrained weithts für passende modelle   
print("Please use the `--pretrained_weights` argument to indicate the path of the checkpoint to evaluate.")
url = None
if arch == "vit_small" and patch_size == 16:
    url = "dino_deitsmall16_pretrain/dino_deitsmall16_pretrain.pth"
elif arch == "vit_small" and patch_size == 8:
    url = "dino_deitsmall8_300ep_pretrain/dino_deitsmall8_300ep_pretrain.pth"  # model used for visualizations in our paper
elif arch == "vit_base" and patch_size == 16:
    url = "dino_vitbase16_pretrain/dino_vitbase16_pretrain.pth"
elif arch == "vit_base" and patch_size == 8:
    url = "dino_vitbase8_pretrain/dino_vitbase8_pretrain.pth"
if url is not None:
    print("Since no pretrained weights have been provided, we load the reference pretrained DINO weights.")
    state_dict = torch.hub.load_state_dict_from_url(url="https://dl.fbaipublicfiles.com/dino/" + url)
    model.load_state_dict(state_dict, strict=True)


In [ ]:
import numpy as np
#komplexe cropfunktion, um später die attentionmaps auf originale bounding boxe zu zuschneiden
def crop_fkt(bbox, img, area, percentile):
  width, height = img.size

  width_edge = bbox[2]*percentile
  height_edge = bbox[3]*percentile

  edge = int(width_edge + height_edge)

  tx = max(bbox[0]-edge, 0)
  ty = max(bbox[1]-edge, 0)
  bx = min(bbox[0]+bbox[2]+edge, width)
  by = min(bbox[1]+bbox[3]+edge, height)

  #find out where edge was added
  left = bbox[0] - tx 
  up = bbox[1] - ty
  right = bx - (bbox[0]+bbox[2])
  down = by - (bbox[1]+bbox[3])

  added_edge = [left, up, right, down]

  area = 280000
  wid = bx - tx
  hei = by - ty
  #print((wid, hei))

  p = np.sqrt(area/(wid*hei))

  imagewidth = int(wid*p) - (int(wid*p)%8)
  imageheight = int(hei*p) - (int(hei*p)%8)

  img_size = (imageheight, imagewidth)

  return (tx, ty, bx, by), added_edge, p, img_size

In [ ]:
import numpy as np
from torchvision.transforms.functional import InterpolationMode

def DINO_fwd(model, image, maskimg, bbox):
  percentile = 0.3
  patch_size = 8 
  area = 230000 

  box, added_edge, p, image_size = crop_fkt(bbox, image, area, percentile)

  crop_img = image.crop(box=box)
  transform = pth_transforms.Compose([
                                    pth_transforms.Resize(image_size),
                                    pth_transforms.ToTensor(),
                                    pth_transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
                                    ])
  crop_img = transform(crop_img)

  w, h = crop_img.shape[1] - crop_img.shape[1] % patch_size, crop_img.shape[2] - crop_img.shape[2] % patch_size
  crop_img = crop_img[:, :w, :h].unsqueeze(0)
  w_featmap = crop_img.shape[-2] // patch_size
  h_featmap = crop_img.shape[-1] // patch_size

  attentions = model.get_last_selfattention(crop_img.to(device))

  nh = attentions.shape[1]

  attentions = attentions[0, :, 0, 1:].reshape(nh, -1)
  attentions = attentions.reshape(nh, w_featmap, h_featmap)
  attentions_transform = pth_transforms.Compose([
                                                  pth_transforms.Resize(image_size, interpolation = InterpolationMode('nearest'))
                                                  ])
  attentions = attentions_transform(attentions)



  attentions = attentions[:, int(added_edge[1]*p): int((added_edge[1] + bbox[3])*p), int(added_edge[0]*p):int((added_edge[0] + bbox[2])*p)]

  crop_mask = maskimg.crop(box=[bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3]])

  mask = np.asarray(crop_mask)
  mask = torch.tensor(mask)

  return attentions, mask

custom dataset

In [ ]:
import os
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader

class NACTI_seg(Dataset):
  def __init__(self, annotations_file, dataset_dir):
    self.annotations_file = annotations_file
    self.coco = COCO(annotations_file)  #for fast reading
    
    self.dataset = dataset_dir
    self.img_ids = self.coco.getImgIds()
    self.ann_ids = self.coco.getAnnIds()


  def __len__(self):
    return len(self.ann_ids)

  def __getitem__(self, idx):
    ann_id = self.ann_ids[idx]
    ann = self.coco.loadAnns([ann_id])
    img_id = ann[0]['image_id']
    img = self.coco.loadImgs([img_id])


    img_path = self.dataset + '/images/' + img[0]['file_name']
    image = None
    mask = None
    bbox = None
    with open(img_path, 'rb') as f:
        image = Image.open(f)
        image = image.convert('RGB')

    mask = self.coco.annToMask(ann[0]) #mask for instance segmentation
    #mask = np.zeros((2000, 2000))
    
    bbox = ann[0]['bbox']
    maskImg = Image.fromarray(mask)
    attentions, mask = DINO_fwd(model, image, maskImg, bbox)

    return attentions, mask,  ann_id

In [ ]:
#tats#chliches berechnen der attentionmaps und speichern dieser

dataset_path = 'PATH_TO_DATASET'

tensor_direc = '/tensors/'


trainset = NACTI_seg('PATH_TO_DATASET_ANNOTATIONS.JSON', dataset_path) #meg_

for i in range(trainset.__len__()):
  attentions, mask, img_id  = trainset.__getitem__(i)

  print(str(i) + ' von ' + str(trainset.__len__()) + '     ' + str(attentions.shape))

  attentions_path = dataset_path + tensor_direc + str(img_id) + '_attention.pt'

  torch.save(attentions.clone(), attentions_path)

In [ ]:
drive.flush_and_unmount()

Drive not mounted, so nothing to flush and unmount.
